In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [45]:
import sys, os
import numpy as np
import scipy 
import tensorflow as tf

import imageio
import gzip
from PIL import Image

import seaborn as sns
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.pyplot as plt
#from tqdm import trange, tqdm

import keras
from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop, Optimizer
from keras.callbacks import Callback
from collections import OrderedDict

from helpers import utils
# from helpers import protocols
# from helpers.keras_utils import LossHistory
# from helpers.optimizers import KOOptimizer

In [4]:
tf.logging.set_verbosity(tf.logging.INFO)

EXTRACT LABELS

In [5]:
# Extract labels from MNIST labels into vector
def extract_labels(filename, num_images):
  with gzip.open(filename) as bytestream:
    bytestream.read(8)
    buf = bytestream.read(1 * num_images)
    labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
  return labels

train_labels = extract_labels("MNIST-data/train-labels-idx1-ubyte.gz", 60000)
eval_labels = extract_labels("MNIST-data/t10k-labels-idx1-ubyte.gz", 10000)
print(np.shape(train_labels))
print(np.shape(eval_labels))

(60000,)
(10000,)


CONSTRUCT DATASETS


In [6]:
# original train
train_original = np.zeros((60000,28,28), dtype=np.float32)
images_original = ["MNIST-processed-training/original/original{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_original)):
    img = np.array(Image.open(images_original[i]))
    train_original[i, :, :] = img
 
print(np.shape(train_original))

# original test
eval_original = np.zeros((10000,28,28), dtype=np.float32)
images2_original = ["MNIST-processed-test/original/test-original{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_original)):
    img = np.array(Image.open(images2_original[i]))
    eval_original[i, :, :] = img
 
print(np.shape(eval_original))

(60000, 28, 28)
(10000, 28, 28)


In [7]:
# ROTATE 90 train
train_rot90 = np.zeros((60000,28,28), dtype=np.float32)
images_rot90 = ["MNIST-processed-training/rot90/rot90{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_rot90)):
    img = np.array(Image.open(images_rot90[i]))
    train_rot90[i, :, :] = img
 
print(np.shape(train_rot90))

# ROTATE 90 test
eval_rot90 = np.zeros((10000,28,28), dtype=np.float32)
images2_rot90 = ["MNIST-processed-test/rot90/test-rot90{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_rot90)):
    img = np.array(Image.open(images2_rot90[i]))
    eval_rot90[i, :, :] = img
 
print(np.shape(eval_rot90))

(60000, 28, 28)
(10000, 28, 28)


In [ ]:
# checkerboard train
train_checkerboard = np.zeros((60000,28,28), dtype=np.float32)
images_checkerboard = ["MNIST-processed-training/checkerboard/fullcheck{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_checkerboard)):
    img = np.array(Image.open(images_checkerboard[i]))
    train_checkerboard[i, :, :] = img
 
print(np.shape(train_checkerboard))

# checkerboard test
eval_checkerboard = np.zeros((10000,28,28), dtype=np.float32)
images2_checkerboard = ["MNIST-processed-test/checkerboard/test-checkerboard{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_checkerboard)):
    img = np.array(Image.open(images2_checkerboard[i]))
    eval_checkerboard[i, :, :] = img
 
print(np.shape(eval_checkerboard))

In [ ]:
# INV train
train_inv = np.zeros((60000,28,28), dtype=np.float32)
images_inv = ["MNIST-processed-training/Inv/inv{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_inv)):
    img = np.array(Image.open(images_inv[i]))
    train_inv[i, :, :] = img
 
print(np.shape(train_inv))

# INV test
eval_inv = np.zeros((10000,28,28), dtype=np.float32)
images2_inv = ["MNIST-processed-test/inv/test-inv{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_inv)):
    img = np.array(Image.open(images2_inv[i]))
    eval_inv[i, :, :] = img
 
print(np.shape(eval_inv))

In [ ]:
# invbot train
train_invbot = np.zeros((60000,28,28), dtype=np.float32)
images_invbot = ["MNIST-processed-training/invbot/invbot{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_invbot)):
    img = np.array(Image.open(images_invbot[i]))
    train_invbot[i, :, :] = img
 
print(np.shape(train_invbot))

# invbot test
eval_invbot = np.zeros((10000,28,28), dtype=np.float32)
images2_invbot = ["MNIST-processed-test/invbot/test-invbot{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_invbot)):
    img = np.array(Image.open(images2_invbot[i]))
    eval_invbot[i, :, :] = img
 
print(np.shape(eval_invbot))

In [ ]:
# fliplr train
train_fliplr = np.zeros((60000,28,28), dtype=np.float32)
images_fliplr = ["MNIST-processed-training/fliplr/fliplr{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_fliplr)):
    img = np.array(Image.open(images_fliplr[i]))
    train_fliplr[i, :, :] = img
 
print(np.shape(train_fliplr))

# fliplr test
eval_fliplr = np.zeros((10000,28,28), dtype=np.float32)
images2_fliplr = ["MNIST-processed-test/fliplr/test-fliplr{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_fliplr)):
    img = np.array(Image.open(images2_fliplr[i]))
    eval_fliplr[i, :, :] = img
 
print(np.shape(eval_fliplr))

In [ ]:
# cutud train
train_cutud = np.zeros((60000,28,28), dtype=np.float32)
images_cutud = ["MNIST-processed-training/cutud/cutUD{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_cutud)):
    img = np.array(Image.open(images_cutud[i]))
    train_cutud[i, :, :] = img
 
print(np.shape(train_cutud))

# cutud test
eval_cutud = np.zeros((10000,28,28), dtype=np.float32)
images2_cutud = ["MNIST-processed-test/cutud/test-cutud{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_cutud)):
    img = np.array(Image.open(images2_cutud[i]))
    eval_cutud[i, :, :] = img
 
print(np.shape(eval_cutud))

In [ ]:
# flipud train
train_flipud = np.zeros((60000,28,28), dtype=np.float32)
images_flipud = ["MNIST-processed-training/flipud/flipud{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_flipud)):
    img = np.array(Image.open(images_flipud[i]))
    train_flipud[i, :, :] = img
 
print(np.shape(train_data))

# flipud test
eval_flipud = np.zeros((10000,28,28), dtype=np.float32)
images2_flipud = ["MNIST-processed-test/flipud/test-flipud{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_flipud)):
    img = np.array(Image.open(images2_flipud[i]))
    eval_flipud[i, :, :] = img
 
print(np.shape(eval_flipud))

PARAMETERS

In [40]:
# # Data params
# input_dim = 784
# output_dim = 10

# input image dimensions
img_rows, img_cols = 28, 28

# # Network params
# n_hidden_units = 2000
# activation_fn = tf.nn.relu

# Optimization params
batch_size = 256
num_classes = 10
epochs = 5 # epochs per task
# learning_rate=1e-3
# xi = 0.1

# Reset optimizer after each age
# reset_optimizer = False

In [34]:
# the data, train and test sets
x_train = train_original
x_test = eval_original
y_train = train_labels
y_test = eval_labels

print('train labels shape:', y_train.shape)
print('test labels shape:', y_test.shape)

train labels shape: (60000,)
test labels shape: (10000,)


In [35]:
# ?
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
# ?
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('train labels shape:', y_train.shape)
print('test labels shape:', y_test.shape)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
train labels shape: (60000, 10)
test labels shape: (10000, 10)


CONSTRUCT MODEL

In [36]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


TRAIN MODEL

In [41]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 175s 3ms/step - loss: 0.0979 - acc: 0.9704 - val_loss: 0.0516 - val_acc: 0.9826
Epoch 2/5
60000/60000 [==============================] - 175s 3ms/step - loss: 0.0740 - acc: 0.9778 - val_loss: 0.0644 - val_acc: 0.9781
Epoch 3/5
60000/60000 [==============================] - 169s 3ms/step - loss: 0.0621 - acc: 0.9816 - val_loss: 0.0353 - val_acc: 0.9886
Epoch 4/5
60000/60000 [==============================] - 166s 3ms/step - loss: 0.0558 - acc: 0.9834 - val_loss: 0.0320 - val_acc: 0.9888
Epoch 5/5
60000/60000 [==============================] - 174s 3ms/step - loss: 0.0475 - acc: 0.9854 - val_loss: 0.0301 - val_acc: 0.9898
Test loss: 0.03006382731956255
Test accuracy: 0.9898


In [42]:
model2 = Sequential()

# Convolutional layer 1 and input layer
model2.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

# Convolutional layer 2
model2.add(Conv2D(64, (3, 3), activation='relu'))

# Pooling layer 1
model2.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout layer with flattening
model2.add(Dropout(0.25))
model2.add(Flatten())

# Dense layer 1 with dropout 
model2.add(Dense(128, activation='relu'))
model2.add(Dropout(0.5))

# Dense layer 2
model2.add(Dense(num_classes, activation='softmax'))

model2.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [43]:
# Save trained model
model.save('trained_model.h5')
# del model

In [65]:
# Reload model
model2 = load_model('trained_model.h5')

In [66]:
# the second data set
x_train2 = train_rot90
x_test2 = eval_rot90
print(x_train2.shape)
print(x_test2.shape)

y_train = train_labels
y_test = eval_labels

(60000, 28, 28)
(10000, 28, 28)


In [67]:
# ?
if K.image_data_format() == 'channels_first':
    x_train2 = x_train2.reshape(x_train2.shape[0], 1, img_rows, img_cols)
    x_test2 = x_test2.reshape(x_test2.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train2 = x_train2.reshape(x_train2.shape[0], img_rows, img_cols, 1)
    x_test2 = x_test2.reshape(x_test2.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
# ?
x_train2 = x_train2.astype('float32')
x_test2 = x_test2.astype('float32')
x_train2 /= 255
x_test2 /= 255
print('x_train2 shape:', x_train2.shape)
print('x_test2 shape:', x_test2.shape)
print(x_train2.shape[0], 'train2 samples')
print(x_test2.shape[0], 'test2 samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('train labels shape:', y_train.shape)
print('test labels shape:', y_test.shape)

x_train2 shape: (60000, 28, 28, 1)
x_test2 shape: (10000, 28, 28, 1)
60000 train2 samples
10000 test2 samples
train labels shape: (60000, 10)
test labels shape: (10000, 10)


In [68]:
# Continue training
model2.fit(x_train2, y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1,
          validation_data=(x_test, y_test))
score1 = model2.evaluate(x_test, y_test, verbose=0)
score2 = model2.evaluate(x_test2, y_test, verbose=0)

print('Original data set')
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

print('Second data set')
print('Test loss:', score2[0])
print('Test accuracy:', score2[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 182s 3ms/step - loss: 0.3183 - acc: 0.9066 - val_loss: 0.9160 - val_acc: 0.7349
Epoch 2/5
60000/60000 [==============================] - 195s 3ms/step - loss: 0.1104 - acc: 0.9673 - val_loss: 1.8853 - val_acc: 0.5208
Epoch 3/5
60000/60000 [==============================] - 177s 3ms/step - loss: 0.0844 - acc: 0.9749 - val_loss: 1.8937 - val_acc: 0.5220
Epoch 4/5
60000/60000 [==============================] - 175s 3ms/step - loss: 0.0693 - acc: 0.9792 - val_loss: 2.0058 - val_acc: 0.5056
Epoch 5/5
60000/60000 [==============================] - 184s 3ms/step - loss: 0.0609 - acc: 0.9819 - val_loss: 2.8436 - val_acc: 0.4279
Original data set
Test loss: 2.843556359291077
Test accuracy: 0.4279
Second data set
Test loss: 0.03505487792348286
Test accuracy: 0.9889


In [ ]:
def cnn_model_fn(features, labels, mode):

    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional layer 1
    conv1 = tf.layers.conv2d(
        inputs = input_layer,
        filters = 32,
        kernel_size=[5,5],
        padding="same",
        activation=tf.nn.relu
    )

    # Pooling 1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional layer 2 and pooling layer
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5,5],
        padding="same",
        activation=tf.nn.relu
    )

    # Pooling 2 with flattening
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)
    pool2_flat=tf.reshape(pool2, [-1, 7 * 7 * 64])

    # Dense layer with dropout 
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout=tf.layers.dropout(inputs=dense, rate=0.4, training=mode==tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dropout, units=10)

    # Generate predictions
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Caluclate loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure training op
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}

    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
# Estimator
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

# Logging predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, 
    every_n_iter=50)

# Our application logic will be added here

In [ ]:
def main(unused_argv):
    # Training on original
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x":train_original},
        y=train_labels,
        batch_size=1000,
        num_epochs=None,
        shuffle=True)
    
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=200,
        hooks=[logging_hook])

    # Evaluation on original
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_original},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    
    eval_results=mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [ ]:
if __name__ == "__main__":
  tf.app.run()